# Demonstrace naší knihovny

## Importy

Nejdříve přidáme složku s naší knihovnou do `sys.path`. Poté ji lze jednoduše importovat.

In [ ]:
import sys
import os
from os import path
import pandas as pd

library_path = path.abspath(path.join(os.getcwd(), "../MH_decipher"))
if library_path not in sys.path:
    sys.path.append(library_path)

import mhdecipher # type: ignore

## Šifrování

In [ ]:
plaintext = "BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS"
key = "DEFGHIJKLMNOPQRSTUVWXYZ_ABC"

ciphertext = mhdecipher.substitute_encrypt(plaintext, key)

print(ciphertext)

## Dešifrování

In [ ]:
ciphertext = "EAOCSRBGQLCYHFHUCSUYQLCPDMCYHFHUQLCPDMCEAOCODVNACFDV"
key = "DEFGHIJKLMNOPQRSTUVWXYZ_ABC"

plaintext = mhdecipher.substitute_decrypt(ciphertext, key)

print(plaintext)

## Vytvoření teoretické bigramové matice

In [ ]:
text = ""
with open("./Krakatit.txt", encoding="utf-8") as f:
    text = f.read()

print(f"Using {len(text)} characters of text to make a transition matrix.")

text = text.upper()

bigrams = mhdecipher.get_bigrams(text)
TM_ref: pd.DataFrame = mhdecipher.transition_matrix(bigrams)

# Visualize the matrix
TM_ref.style.format().background_gradient(axis=None)

## Provedení kryptoanalýzy na zašifrovaném textu

Kód předpokládá vygenerovanou bigramovou matici z minulé sekce.

In [ ]:
ciphertext = "EAOCSRBGQLCYHFHUCSUYQLCPDMCYHFHUQLCPDMCEAOCODVNACFDV"
iter = 20_000
start_key = "ABCDEFGHIJKLMNOPQRSTUVWXYZ_"

current_key, best_decrypted_text, p_current = mhdecipher.prolom_substitute(ciphertext, TM_ref, iter, start_key)

print(f"Current key: {current_key}")
print(f"Best decrypted text: {best_decrypted_text}")
print(f"Current P: {p_current}")

## Test: get_bigrams

In [ ]:
bigrams = mhdecipher.get_bigrams("KRYPTOSYSTEM")
print(bigrams)

assert bigrams == ["KR", "RY", "YP", "PT", "TO", "OS", "SY", "YS", "ST", "TE", "EM"]  # test expected value

## Test: plausibility

Kód předpokládá vygenerovanou bigramovou matici výše.

In [ ]:
plausibility = mhdecipher.plausibility("BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS", TM_ref)
print(plausibility)

plausibility = mhdecipher.plausibility("EAOCSRBGQLCYHFHUCSUYQLCPDMCYHFHUQLCPDMCEAOCODVNACFDV", TM_ref)
print(plausibility)